In [1]:
# importando bibliotecas necessárias
import pandas as pd
import requests
import hashlib
import re

# importando funções de limpeza e transformação 
from cnj_dados import cnj_dados, encontrar_maior_cod_item_pai
from harmonizacao_str import limpar_assunto, remover_acentos_e_maiusculo,identificar_cidade,limpar_nome_municipio,identificar_capital
from preenche_codigos_cnj import preencher_codigos, preencher_codigos_hierarquicos

In [2]:
# Definindo tipagem para principais colunas
dict_type_cnpj = {
    'cnpj':'str'
}

# Lendo json em dataframe para manipulação dos dados
df_bq_results = pd.read_json('../raw_data/bq-results-20240515-184938-1715799987947.json',
                             lines=True,dtype=dict_type_cnpj)
df_bq_results.shape

(20000, 18)

Primeiros tratamentos no dataframe

In [3]:
# Adicionando zeros à esquerda para normalizar todos os CNPJs para 14 caracteres
df_bq_results['cnpj'] = df_bq_results['cnpj'].str.zfill(14)

In [4]:
# Renomeando colunas para manter o padrao snake case dos demais arquivos
df_bq_results.rename(columns={
    'grauProcesso':'grau_processo', 
    'dataDecisao':'data_decisao',
    'dataEncerramento':'data_encerramento', 
    'ultimoEstado':'ultimo_estado', 
    'orgaoJulgador':'orgao_julgador',
    'citacaoTipo':'citacao_tipo', 
    'unidadeOrigem':'municipio',
    'assuntosCNJ':'assuntos_cnj',
    'valorCausa':'valor_causa', 
    'valorPredicaoCondenacao':'valor_predicao_condenacao'
},inplace=True)

In [5]:
# Função para contar tamanho do dicionario contido nas colunas
def count_len_neoway(col):
    return len(col) if col else 0

# Aplicando função às colunas que possuem dict
df_bq_results['contagem_partes'] = df_bq_results['partes'].apply(count_len_neoway)
df_bq_results['contagem_assuntos_cnj'] = df_bq_results['assuntos_cnj'].apply(count_len_neoway)

In [6]:
# Dividindo a coluna 'data_decisao' em 'data' e 'hora'
df_bq_results[['dt_data_decisao', 'hora_decisao']] = df_bq_results['data_decisao'].str.split('T', expand=True)
# Removendo 'Z' da coluna 'hora' para obter apenas o tempo
df_bq_results['hora_decisao'] = df_bq_results['hora_decisao'].str.replace('Z', '')

# Dividindo a coluna 'data_encerramento' em 'data' e 'hora'
df_bq_results[['dt_data_encerramento', 'hora_encerramento']] = df_bq_results['data_encerramento'].str.split('T', expand=True)
# Removendo 'Z' da coluna 'hora' para obter apenas o tempo
df_bq_results['hora_encerramento'] = df_bq_results['hora_encerramento'].str.replace('Z', '')

In [7]:
# expandido orgao em colunas
df_bq_results[['orgao_julgador_orgao', 'orgao_julgador_vara', 'orgao_julgador_uf']] = df_bq_results['orgao_julgador'].str.split(' - ', expand=True, n=2)

In [8]:
# drop de colunas desnecessárias (as que foram tratadas acima e as redundantes)
df_bq_results.drop(columns=['comarca','orgao_julgador_vara',
                            'orgao_julgador_uf','orgao_julgador',
                           'data_decisao','data_encerramento'],
                   inplace=True                  
                  )

In [9]:
# drop de linhas duplicadas
df_bq_results = df_bq_results.drop_duplicates(subset=['cnpj', 'area', 'grau_processo', 'julgamento', 'uf', 'tribunal',
       'ultimo_estado', 'citacao_tipo', 'municipio', 'juiz', 'valor_causa', 'valor_predicao_condenacao', 'contagem_partes',
       'contagem_assuntos_cnj', 'orgao_julgador_orgao', 'dt_data_decisao',
       'hora_decisao', 'dt_data_encerramento', 'hora_encerramento'])

In [10]:
# Função para criar um ID único para cada processo
def gerar_codigo_unico(linha):
    texto = ''.join(linha.astype(str))
    return hashlib.md5(texto.encode()).hexdigest()

# Aplicando a função a cada linha do DataFrame
df_bq_results['cd_processo'] = df_bq_results.apply(gerar_codigo_unico, axis=1)

In [11]:
#reordenando colunas para id -> cd_processo ser a primeira
df_bq_results = df_bq_results[['cd_processo','cnpj', 'area', 'grau_processo', 'julgamento', 'uf', 'tribunal',
       'ultimo_estado', 'citacao_tipo', 'municipio', 'juiz', 'assuntos_cnj',
       'partes', 'valor_causa', 'valor_predicao_condenacao', 'contagem_partes',
       'contagem_assuntos_cnj', 'orgao_julgador_orgao', 'dt_data_decisao',
       'hora_decisao', 'dt_data_encerramento', 'hora_encerramento']]

### Daqui pra baixo o df será dividido em partes que serão tratadas isolodamentes.

- judicializacao geral -> Remoção de colunas que possuem dicionário, que serão tratadas separadaemnete 


- partes -> Dimensão das partes envolvidas em cada processo. Contém as colunas cd_processo e coluna com dicionario das partes


- assuntos cnj -> Dimensão dos assuntos do  CNJ contidos em cada processo. Contém as colunas cd_processo e coluna com dicionario dos assuntos do cnj

#### Judicialização geral

In [12]:
# Data frame de judicialização geral
df_judicializacao = df_bq_results.drop(columns={
    'partes'
    ,'assuntos_cnj'
})

In [13]:
# Tratamento da coluna município com as funções definidas e chamadas na importação
df_judicializacao['municipio'] = df_judicializacao['municipio'].fillna('Nao Identificado')
df_judicializacao['municipio'] = df_judicializacao['municipio'].astype(str)

df_judicializacao['municipio'] = df_judicializacao['municipio'].apply(limpar_nome_municipio)
df_judicializacao['municipio'] = df_judicializacao['municipio'].apply(identificar_capital)
df_judicializacao['municipio'] = df_judicializacao['municipio'].apply(identificar_cidade)

In [14]:
# Tratamento pontual de alguns valores da coluna município  que não foram contemplados no tratamento
# Substituindo "Turmas Recursais Estado Do Rio Grande Do Sul" por "Porto Alegre"
df_judicializacao.loc[
    df_judicializacao["municipio"] == "Turmas Recursais Estado Do Rio Grande Do Sul",
    "municipio"] = "Porto Alegre"

# Substituindo "Leal Fagundes Brasilia" por "Brasilia"
df_judicializacao.loc[
    df_judicializacao["municipio"] == "Leal Fagundes Brasilia",
    "municipio"] = "Brasilia"

# Substituindo "Juizados Especiais De Rio Branco" por "Rio Branco"
df_judicializacao.loc[
    df_judicializacao["municipio"] == "Juizados Especiais De Rio Branco",
    "municipio"] = "Rio Branco"


In [15]:
# Tratamento pontual de alguns valores da coluna uf que não foram contemplados pelas funções
# Substituindo "R5" por "CE"
df_judicializacao.loc[
    df_judicializacao["uf"] == "R5",
    "uf"] = "CE"

# Substituindo "R1" por "MG"
df_judicializacao.loc[
    df_judicializacao["uf"] == "R1",
    "uf"] = "MG"

In [16]:
# Carga em dbt/seed
df_judicializacao.to_csv('../../neoway_case/seeds/empresas_judicializacao_geral.csv',sep=',',index=False)

#### dimensão das partes envolvidas nos processos

In [17]:
# Data frame das partes
df_partes = df_bq_results[df_bq_results.contagem_partes>0][['cd_processo','partes']]

In [18]:
# Expandindo a coluna 'partes' para que cada elemento da lista crie uma nova linha
df_partes = df_partes.explode('partes')

# Extraindo 'nomeNormalizadoNeoway' e 'polo' de cada dicionário em novas colunas
df_partes['nomeNormalizadoNeoway'] = df_partes['partes'].apply(lambda x: x['nomeNormalizadoNeoway'] if pd.notnull(x) else None)
df_partes['polo'] = df_partes['partes'].apply(lambda x: x['polo'] if pd.notnull(x) else None)

# Removendo a coluna 'partes' original, agora desnecessária
df_partes.drop(columns=['partes'], inplace=True)

In [19]:
# Dropo de partes duplicadas em cada processo
df_partes = df_partes.drop_duplicates()

In [20]:
# Renomeando coluna para manter o padrão snake case
df_partes.rename(columns={
    'nomeNormalizadoNeoway':'nome_normalizado_neoway'
},inplace=True)

In [21]:
# Carga em dbt/seed
df_partes.to_csv('../../neoway_case/seeds/empresas_judicializacao_partes.csv',sep=',',index=False)

#### dimensão dos assuntos do CNJ de cada processo

- a coluna codigoLocal foi ignorada propositalmente devido ao fato de não ser relevante para o prosseguimento do projeto

In [22]:
# Data Frame de assuntos do CNJ
df_assuntos_cnj = df_bq_results[df_bq_results.contagem_assuntos_cnj>0][['cd_processo','assuntos_cnj']]

In [23]:
# Expandindo a coluna 'assuntos_cnj' para que cada elemento da lista crie uma nova linha
df_assuntos_cnj = df_assuntos_cnj.explode('assuntos_cnj')

# Extraindo 'titulo' e 'codigoCNJ' de cada dicionário em novas colunas
df_assuntos_cnj['titulo'] = df_assuntos_cnj['assuntos_cnj'].apply(lambda x: x.get('titulo') if isinstance(x, dict) else None)
df_assuntos_cnj['codigo_cnj'] = df_assuntos_cnj['assuntos_cnj'].apply(lambda x: x.get('codigoCNJ') if isinstance(x, dict) else None)
# df_assuntos_cnj['codigo_local'] = df_assuntos_cnj['assuntos_cnj'].apply(lambda x: x.get('codigoLocal') if isinstance(x, dict) else None)

# Removendo a coluna 'assuntos_cnj' original, agora desnecessária
df_assuntos_cnj.drop(columns=['assuntos_cnj'], inplace=True)

In [24]:
# Drop de assuntos duplicados em cada processo
df_assuntos_cnj = df_assuntos_cnj.drop_duplicates()

In [25]:
# Tratamento de coluna de codigos_cnj preenchendo vazios e mudando tipagem
df_assuntos_cnj.codigo_cnj = df_assuntos_cnj.codigo_cnj.fillna(0).astype(int)

In [26]:
# Preenchendo valores vazios da coluna titulo
df_assuntos_cnj.titulo = df_assuntos_cnj.titulo.fillna('')

# Tratamento da coluna titulo com função de harmonização definidas e chamadas na importação
df_assuntos_cnj['titulo'] = df_assuntos_cnj['titulo'].apply(remover_acentos_e_maiusculo)

##### Dados externos CNJ

- A fim de harmonizar e enriquecer os dados da tabela de assuntos CNJ, será utilizada uma API dos assuntos do CNJ para melhorar qualidade de preenchimento da tabela 

In [27]:
# Requisação para gerar dataframe com codigos do CNJ
df_cnj = cnj_dados()

In [28]:
# Aplicando função para gerar uma nova coluna com o codigo_pai maximo ao subir na hierarquia de assuntos
df_cnj['maior_cod_item_pai'] = df_cnj['cod_item'].apply(lambda x: encontrar_maior_cod_item_pai(df_cnj, x))

In [29]:
# Tratamento de colunas a fim de harmonizar os dados e manter mesma harmonização de df_assuntos_cnj 
df_cnj.nome = df_cnj.nome.apply(remover_acentos_e_maiusculo)
df_cnj.cod_item_pai = df_cnj.cod_item_pai.fillna(0).astype(int)

In [30]:
# Carga em dbt/seed
df_cnj.to_csv('../../neoway_case/seeds/codigos_cnj.csv',
                      index=False,sep=',')

##### Enriquecimento de dados da tabela df_assuntos_cnj

In [31]:
'''
    Função para preencher código CNJ quando o assunto de  dos dois dataframe é 
    exatamente igual e só há um único código correspondente em df_cnj para esse assunto.
    Dessa forma, se garante o preenchimento do código correto e evita-se relacionamentos 1:N
'''

# Sem filtro de situação
df_assuntos_cnj = preencher_codigos(df_cnj, df_assuntos_cnj)
# Com filtro de situação 'A' (Ativo)
df_assuntos_cnj = preencher_codigos(df_cnj, df_assuntos_cnj, situacao='A')
# Com filtro de situação 'I' (Inativo)
df_assuntos_cnj = preencher_codigos(df_cnj, df_assuntos_cnj, situacao='I')

In [32]:
'''
    Função para preencher código CNJ quando o assunto de dos dois dataframe é 
    exatamente igual, mas há mais de um código para esse assunto em df_cnj.
    Nesse caso, se sobe na hierarquia de assuntos e para verificar se 
    o código da hierarquia máxima é único. Se sim, é aplicado o código desse nível.
    Dessa forma, se garante o preenchimento do código correto e evita-se relacionamentos 1:N
'''

# Sem filtro de situação
df_assuntos_cnj = preencher_codigos_hierarquicos(df_cnj, df_assuntos_cnj)
# Com filtro de situação 'A' (Ativo)
df_assuntos_cnj = preencher_codigos_hierarquicos(df_cnj, df_assuntos_cnj, situacao='A')
# Com filtro de situação 'I' (Inativo)
df_assuntos_cnj = preencher_codigos_hierarquicos(df_cnj, df_assuntos_cnj, situacao='I')

In [33]:
df_assuntos_cnj[df_assuntos_cnj.codigo_cnj==0].shape

(6968, 3)

preenchendo os codigos mais comumns dos titulos que possuem valor = 0 

In [34]:
'''
    Bloco de código para preencher os valores ainda nulos com o código mais frequente
    do próprio df_assuntos_cnj quando houver correspondência do título nessa tabela
'''

lst_assuntos = df_assuntos_cnj[df_assuntos_cnj.codigo_cnj==0].titulo.unique()
# Para cada título, encontrar o valor mais comum (mode) do codigo_cnj
for titulo in lst_assuntos:
    # Encontrar o mode do codigo_cnj para o título específico
    mode_codigo = df_assuntos_cnj[
        (df_assuntos_cnj.titulo == titulo) & (df_assuntos_cnj.codigo_cnj != 0)
    ]['codigo_cnj'].mode()
    
    # Se houver um valor mais comum, preenchê-lo nos registros onde codigo_cnj é 0
    if not mode_codigo.empty:
        df_assuntos_cnj.loc[
            (df_assuntos_cnj.titulo == titulo) & (df_assuntos_cnj.codigo_cnj == 0), 'codigo_cnj'
        ] = mode_codigo[0]

df_assuntos_cnj[df_assuntos_cnj.codigo_cnj==0].shape

(5252, 3)

In [35]:
'''
    Aplicando função de limpeza e harmonização nos títulos que ainda estão sem um código cnj.
    As etapas acima serão repetidas na exata mesma ordem depois desse tratamento feito.
'''

mask = (df_assuntos_cnj.codigo_cnj == 0)
df_assuntos_cnj.loc[mask,'titulo'] = df_assuntos_cnj.loc[mask, 'titulo'].apply(limpar_assunto)


In [36]:
'''
    Função para preencher código CNJ quando o assunto de  dos dois dataframe é 
    exatamente igual e só há um único código correspondente em df_cnj para esse assunto.
    Dessa forma, se garante o preenchimento do código correto e evita-se relacionamentos 1:N
'''

# Sem filtro de situação
df_assuntos_cnj = preencher_codigos(df_cnj, df_assuntos_cnj)
# Com filtro de situação 'A' (Ativo)
df_assuntos_cnj = preencher_codigos(df_cnj, df_assuntos_cnj, situacao='A')
# Com filtro de situação 'I' (Inativo)
df_assuntos_cnj = preencher_codigos(df_cnj, df_assuntos_cnj, situacao='I')

In [37]:
df_assuntos_cnj[df_assuntos_cnj.codigo_cnj==0].shape

(3228, 3)

In [38]:
'''
    Função para preencher código CNJ quando o assunto de dos dois dataframe é 
    exatamente igual, mas há mais de um código para esse assunto em df_cnj.
    Nesse caso, se sobe na hierarquia de assuntos e para verificar se 
    o código da hierarquia máxima é único. Se sim, é aplicado o código desse nível.
    Dessa forma, se garante o preenchimento do código correto e evita-se relacionamentos 1:N
'''
# Sem filtro de situação
df_assuntos_cnj = preencher_codigos_hierarquicos(df_cnj, df_assuntos_cnj)
# Com filtro de situação 'A' (Ativo)
df_assuntos_cnj = preencher_codigos_hierarquicos(df_cnj, df_assuntos_cnj, situacao='A')
# Com filtro de situação 'I' (Inativo)
df_assuntos_cnj = preencher_codigos_hierarquicos(df_cnj, df_assuntos_cnj, situacao='I')

In [39]:
df_assuntos_cnj[df_assuntos_cnj.codigo_cnj==0].shape

(2485, 3)

In [40]:
'''
    Bloco de código para preencher os valores ainda nulos com o código mais frequente
    do próprio df_assuntos_cnj quando houver correspondência do título nessa tabela
'''

lst_assuntos = df_assuntos_cnj[df_assuntos_cnj.codigo_cnj==0].titulo.unique()
# Para cada título, encontrar o valor mais comum (mode) do codigo_cnj
for titulo in lst_assuntos:
    # Encontrar o mode do codigo_cnj para o título específico
    mode_codigo = df_assuntos_cnj[
        (df_assuntos_cnj.titulo == titulo) & (df_assuntos_cnj.codigo_cnj != 0)
    ]['codigo_cnj'].mode()
    
    # Se houver um valor mais comum, preenchê-lo nos registros onde codigo_cnj é 0
    if not mode_codigo.empty:
        df_assuntos_cnj.loc[
            (df_assuntos_cnj.titulo == titulo) & (df_assuntos_cnj.codigo_cnj == 0), 'codigo_cnj'
        ] = mode_codigo[0]

df_assuntos_cnj[df_assuntos_cnj.codigo_cnj==0].shape

(2342, 3)

In [41]:
# removendo titulos apos todos os tratamentos e dropando assuntos duplicados por processo
df_assuntos_cnj = df_assuntos_cnj[['cd_processo','codigo_cnj']].drop_duplicates()

In [42]:
# Carga em dbt/seed
df_assuntos_cnj.to_csv('../../neoway_case/seeds/empresas_judicializacao_assuntos_cnj.csv',
                      index=False,sep=',')